In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import re
import os

In [2]:
base_dir = r"D:\Path\to\the\directory\where\the\extracted_mimic_iii_data\is\stored"

Load Core Tables

In [3]:
patients = pd.read_csv(f"{base_dir}\PATIENTS.csv")
admissions = pd.read_csv(f"{base_dir}\ADMISSIONS.csv")
icustays = pd.read_csv(f"{base_dir}\ICUSTAYS.csv")

In [4]:
# Parse cols as date, convert bad dates to NaT (dropped later)
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')

In [5]:
# Merging minimal columns first
adm = admissions.merge(patients[['SUBJECT_ID', 'GENDER', 'DOB']], on='SUBJECT_ID', how='left')

In [6]:
# Drop rows with missing critical dates
adm = adm.dropna(subset=['ADMITTIME', 'DOB'])

In [7]:
# Computing age by components
admit_y = adm['ADMITTIME'].dt.year
admit_m = adm['ADMITTIME'].dt.month
admit_d = adm['ADMITTIME'].dt.day

In [8]:
dob_y = adm['DOB'].dt.year
dob_m = adm['DOB'].dt.month
dob_d = adm['DOB'].dt.day

In [9]:
age = admit_y - dob_y
before_bday = (admit_m < dob_m) | ((admit_m == dob_m) & (admit_d < dob_d))
age = age - before_bday.astype(int)

In [10]:
# Age bounds 0-120 years
# Combine (de-identify) ages above 89
age = age.where((age >= 0) & (age <= 120))
age = age.clip(upper=89)

In [11]:
adm['AGE'] = age.astype('float')

In [12]:
icustays['HADM_ID'] = icustays['HADM_ID'].astype(str)
adm['HADM_ID'] = adm['HADM_ID'].astype(str)

In [13]:
adm_icu = adm.merge(
    icustays[['SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','OUTTIME','LOS']],
    on=['SUBJECT_ID','HADM_ID'],
    how='inner'
)

In [14]:
# Keeping only the important columns
adm_icu = adm_icu[['SUBJECT_ID','HADM_ID','ICUSTAY_ID','AGE','GENDER','ADMISSION_TYPE', 'DIAGNOSIS',
                   'INTIME','OUTTIME','LOS','HOSPITAL_EXPIRE_FLAG']]

Map lab names to ITEMIDs from D_LABITEMS

In [15]:
dlabs = pd.read_csv(f"{base_dir}\D_LABITEMS.csv")

In [16]:
# Normalize text
dlabs['LABEL_NORM'] = dlabs['LABEL'].str.upper().str.strip()
dlabs['FLUID'] = dlabs['FLUID'].str.upper().str.strip()
dlabs['CATEGORY'] = dlabs['CATEGORY'].str.upper().str.strip()

In [17]:
# Canonical lab names and simple label patterns
lab_patterns = {
    'GLUCOSE':    r'\bGLUCOSE\b',
    'CREATININE': r'\bCREATININE\b',
    'SODIUM':     r'\bSODIUM\b',
    'POTASSIUM':  r'\bPOTASSIUM\b',
    'HEMATOCRIT': r'\bHEMATOCRIT\b',
    'BUN':        r'\bUREA\s*NITROGEN\b|\bBLOOD\s*UREA\s*NITROGEN\b|\bBUN\b',
    'LACTATE':    r'\bLACTATE\b'
}

In [18]:
# Filter to avoid mapping urine, etc. to blood glucose
dlabs_filt = dlabs[dlabs['FLUID'] == 'BLOOD'].copy()

In [19]:
# Build ITEMID to canonical TEST mapping
lab_rows = []
for canon, pat in lab_patterns.items():
    hits = dlabs_filt[dlabs_filt['LABEL_NORM'].str.contains(pat, regex=True, na=False)].copy()
    if not hits.empty:
        hits['LAB_TEST'] = canon
        lab_rows.append(hits[['ITEMID','LABEL','CATEGORY','LAB_TEST']])

lab_dict_df = pd.concat(lab_rows, ignore_index=True).drop_duplicates(subset=['ITEMID','LAB_TEST'])
lab_itemids = lab_dict_df[['ITEMID','LAB_TEST']].drop_duplicates()

lab_dict_df.head()

,ITEMID,LABEL,CATEGORY,LAB_TEST
0,50809,Glucose,BLOOD GAS,GLUCOSE
1,50931,Glucose,CHEMISTRY,GLUCOSE
2,51529,Estimated Actual Glucose,CHEMISTRY,GLUCOSE
3,50912,Creatinine,CHEMISTRY,CREATININE
4,50824,"Sodium, Whole Blood",BLOOD GAS,SODIUM


Keep only consistent units per test

In [20]:
# Defining preferred units per lab
preferred_units = {
    'GLUCOSE': 'mg/dL',
    'CREATININE': 'mg/dL',
    'SODIUM': 'mEq/L',       # Special case (mmol/L)
    'POTASSIUM': 'mEq/L',    # Special case (mmol/L)
    'HEMATOCRIT': '%',
    'BUN': 'mg/dL',
    'LACTATE': 'mmol/L'
}

In [21]:
# For Na and K, mmol/L and mEq/L are numerically equvivalent, therefore, both allowed 
def unit_ok(row):
    test = row['LAB_TEST']
    u = str(row.get('VALUEUOM','')).strip()
    if test in ('SODIUM','POTASSIUM'):
        return u in ('mEq/L','mmol/L')
    return (preferred_units.get(test) == u)

In [22]:
# ITEMID to LAB_TEST mapping
lab_itemids = lab_dict_df[['ITEMID', 'LAB_TEST']].drop_duplicates()

Read LABEVENTS in chunks and aggrate for the first 24 hours

In [23]:
# Unify key dtypes
adm_icu['SUBJECT_ID'] = adm_icu['SUBJECT_ID'].astype('Int64')
adm_icu['HADM_ID']    = adm_icu['HADM_ID'].astype('string')

In [24]:
# Building a 24h Window per ICU stay
adm_icu['INTIME'] = pd.to_datetime(adm_icu['INTIME'], errors='coerce')
adm_icu['INTIME_24H'] = adm_icu['INTIME'] + pd.Timedelta(hours=24)

In [25]:
icu_keys = adm_icu[['SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','INTIME_24H']].copy()

In [26]:
# Join on SUBJECT_ID, HADM_ID to get ICU window, then filter by CHARTTIME window
use_cols = ['SUBJECT_ID','HADM_ID','ITEMID','CHARTTIME','VALUENUM','VALUEUOM']
lab_chunks = pd.read_csv(f"{base_dir}\LABEVENTS.csv", usecols=use_cols,
    chunksize=1_000_000,
    low_memory=False,
    dtype={
        'SUBJECT_ID': 'Int64',   # nullable integer
        'HADM_ID': 'string',     # match adm_icu which uses string
        'ITEMID': 'Int64',
        'VALUEUOM': 'string'
    }
)

In [27]:
lab_itemids_map = lab_itemids.set_index('ITEMID')['LAB_TEST']

In [28]:
# Allowed units per test
allowed_units = {
    'GLUCOSE': {'mg/dL'},
    'CREATININE': {'mg/dL'},
    'SODIUM': {'mEq/L', 'mmol/L'},
    'POTASSIUM': {'mEq/L', 'mmol/L'},
    'HEMATOCRIT': {'%'},
    'BUN': {'mg/dL'},
    'LACTATE': {'mmol/L'}
}

In [29]:
# Process LABEVENTS in chunks
collected = []

for chunk in lab_chunks:
    chunk['CHARTTIME'] = pd.to_datetime(chunk['CHARTTIME'], errors='coerce')
    # Keep only necessary ITEMIDs
    chunk = chunk[chunk['ITEMID'].isin(lab_itemids_map.index)]
    if chunk.empty:
        continue

    # Attach canonical lab name
    chunk['LAB_TEST'] = chunk['ITEMID'].map(lab_itemids_map)

    # Join to ICU stays to get time window
    key_cols = ['SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','INTIME_24H']
    icu_keys = adm_icu[key_cols]
    merged = chunk.merge(icu_keys, on=['SUBJECT_ID','HADM_ID'], how='inner')

    # Keep only first 24h window rows
    win = merged[(merged['CHARTTIME'] >= merged['INTIME']) & (merged['CHARTTIME'] <= merged['INTIME_24H'])]

    # Keep only numeric values
    win = win[pd.to_numeric(win['VALUENUM'], errors='coerce').notna()]
    win['VALUENUM'] = win['VALUENUM'].astype(float)

    if not win.empty:
        collected.append(win[['ICUSTAY_ID','LAB_TEST','VALUENUM']])

In [30]:
# Check datatypes
print(chunk[['SUBJECT_ID','HADM_ID']].dtypes)
print(icu_keys[['SUBJECT_ID','HADM_ID']].dtypes)

SUBJECT_ID             Int64
HADM_ID       string[python]
dtype: object
SUBJECT_ID             Int64
HADM_ID       string[python]
dtype: object


In [31]:
# Concatenate all filtered rows
if collected:
    labs_24h = pd.concat(collected, ignore_index=True)
else:
    labs_24h = pd.DataFrame(columns=['ICUSTAY_ID','LAB_TEST','VALUENUM'])

In [32]:
# Aggregate to mean, min, max per lab per ICU stay
lab_agg = labs_24h.groupby(['ICUSTAY_ID','LAB_TEST'])['VALUENUM'].agg(['mean','min','max']).reset_index()

In [33]:
# Pivot to columns
lab_wide = lab_agg.pivot(index='ICUSTAY_ID', columns='LAB_TEST')
lab_wide.columns = [f"{lab}_{stat}" for stat, lab in lab_wide.columns]
lab_wide = lab_wide.reset_index()

In [34]:
lab_wide.head()

,ICUSTAY_ID,BUN_mean,CREATININE_mean,GLUCOSE_mean,HEMATOCRIT_mean,LACTATE_mean,POTASSIUM_mean,SODIUM_mean,BUN_min,CREATININE_min,...,LACTATE_min,POTASSIUM_min,SODIUM_min,BUN_max,CREATININE_max,GLUCOSE_max,HEMATOCRIT_max,LACTATE_max,POTASSIUM_max,SODIUM_max
0,200001,83.0,2.70,87.0,23.600000,105.20,4.45,139.000000,83.0,2.7,...,1.4,4.3,139.0,83.0,2.7,87.0,23.6,209.0,4.6,139.0
1,200003,19.5,0.85,124.5,33.050000,56.95,3.15,142.500000,19.0,0.7,...,2.6,3.1,141.0,20.0,1.0,159.0,35.0,216.0,3.2,144.0
2,200006,14.0,0.80,71.0,25.050000,NaN,3.50,137.000000,14.0,0.8,...,NaN,3.5,137.0,14.0,0.8,71.0,25.3,NaN,3.5,137.0
3,200007,9.0,0.80,225.0,38.100000,NaN,3.85,136.500000,8.0,0.8,...,NaN,3.8,135.0,10.0,0.8,233.0,38.1,NaN,3.9,138.0
4,200009,15.5,0.50,113.6,27.914286,2.00,4.54,138.833333,15.0,0.5,...,0.7,3.8,135.0,16.0,0.5,161.0,37.0,2.7,5.6,143.0


Merge into the main dataframe

In [35]:
# Merge lab features into the ICU cohort
cohort = adm_icu.merge(lab_wide, on='ICUSTAY_ID', how='left')

In [36]:
cohort.shape, cohort.columns.tolist()[:15]

((61532, 33),
 ['SUBJECT_ID',
  'HADM_ID',
  'ICUSTAY_ID',
  'AGE',
  'GENDER',
  'ADMISSION_TYPE',
  'DIAGNOSIS',
  'INTIME',
  'OUTTIME',
  'LOS',
  'HOSPITAL_EXPIRE_FLAG',
  'INTIME_24H',
  'BUN_mean',
  'CREATININE_mean',
  'GLUCOSE_mean'])

In [37]:
# Feature coverage (which labs are commonly used)
feat_cols = [c for c in cohort.columns if c.endswith(('_mean','_min','_max'))]
coverage = cohort[feat_cols].notna().mean().sort_values(ascending=False)
print(coverage.head(10))
print(coverage.tail(10))

# Label balance
cohort['HOSPITAL_EXPIRE_FLAG'].value_counts(normalize=True)

# Save clean snapshot
cohort.to_csv(f"{base_dir}\MERGED\cohort_labs24h.csv", index=False)

HEMATOCRIT_min     0.910437
HEMATOCRIT_mean    0.910437
HEMATOCRIT_max     0.910437
POTASSIUM_min      0.874537
POTASSIUM_max      0.874537
POTASSIUM_mean     0.874537
SODIUM_min         0.871156
SODIUM_max         0.871156
SODIUM_mean        0.871156
GLUCOSE_min        0.841562
dtype: float64
GLUCOSE_mean       0.841562
CREATININE_min     0.840603
CREATININE_mean    0.840603
CREATININE_max     0.840603
BUN_min            0.840164
BUN_max            0.840164
BUN_mean           0.840164
LACTATE_min        0.445378
LACTATE_mean       0.445378
LACTATE_max        0.445378
dtype: float64


CHARTEVENT Operations (mapping D_ITEMS first)

In [38]:
# Load dictionary for charted items
ditems = pd.read_csv(f"{base_dir}\D_ITEMS.csv")
ditems['LABEL_NORM'] = ditems['LABEL'].str.upper().str.strip()
ditems['LINKSTO'] = ditems['LINKSTO'].str.upper().str.strip()

In [39]:
# Keep only items that link to CHARTEVENTS
dchart = ditems[ditems['LINKSTO'] == 'CHARTEVENTS'].copy()

In [40]:
# Map variations
vital_patterns = {
    'HEART_RATE': r'\bHEART\s*RATE\b',
    'RESP_RATE':  r'\bRESPIRATORY\s*RATE\b',
    # Merge invasive and noninvasive BP sources
    'SBP':        r'\b(?:NON\s*INVASIVE|NIBP|ARTERIAL|ABP).*\bSYSTOLIC\b|\bSYSTOLIC\s*BLOOD\s*PRESSURE\b',
    'DBP':        r'\b(?:NON\s*INVASIVE|NIBP|ARTERIAL|ABP).*\bDIASTOLIC\b|\bDIASTOLIC\s*BLOOD\s*PRESSURE\b',
    'MAP':        r'\b(?:NON\s*INVASIVE|NIBP|ARTERIAL|ABP).*\bMEAN\b|\bMEAN\s*BLOOD\s*PRESSURE\b',
    # Temperature in C or F (F coverted to C later)
    'TEMP_C':     r'\bTEMP(?:ERATURE)?\b.*\bC(?:ELSIUS)?\b',
    'TEMP_F':     r'\bTEMP(?:ERATURE)?\b.*\bF(?:AHRENHEIT)?\b',
    # Pulse oximetry variants
    'SPO2':       r'\b(?:SPO2|OXYGEN\s*SATURATION|O2\s*SATURATION)\b'
}

In [41]:
rows = []
for canon, pat in vital_patterns.items():
    hits = dchart[dchart['LABEL_NORM'].str.contains(pat, regex=True, na=False)].copy()
    if not hits.empty:
        hits['VITAL'] = canon
        rows.append(hits[['ITEMID','LABEL','VITAL']])

In [42]:
vital_map_df = pd.concat(rows, ignore_index=True).drop_duplicates(subset=['ITEMID','VITAL'])
vital_itemids = vital_map_df[['ITEMID','VITAL']].drop_duplicates()

In [43]:
vital_itemids.groupby('VITAL')['ITEMID'].nunique()

VITAL
DBP            5
HEART_RATE     6
MAP            9
RESP_RATE      6
SBP            5
SPO2          15
TEMP_C         9
TEMP_F         5
Name: ITEMID, dtype: int64

In [44]:
vital_map_df[vital_map_df['VITAL'].isin(['SBP','DBP','MAP'])].head(20)

,ITEMID,LABEL,VITAL
12,6,ABP [Systolic],SBP
13,51,Arterial BP [Systolic],SBP
14,6701,Arterial BP #2 [Systolic],SBP
15,220050,Arterial Blood Pressure systolic,SBP
16,220179,Non Invasive Blood Pressure systolic,SBP
17,8364,ABP [Diastolic],DBP
18,8368,Arterial BP [Diastolic],DBP
19,8555,Arterial BP #2 [Diastolic],DBP
20,220051,Arterial Blood Pressure diastolic,DBP
21,220180,Non Invasive Blood Pressure diastolic,DBP


Aggregate 24h Vitals for CHARTEVENTS

In [45]:
# ICU key frame and 24h window
icu_keys = adm_icu[['ICUSTAY_ID','INTIME','INTIME_24H']].copy()

In [46]:
# Parse in 1M chunks
use_cols = ['SUBJECT_ID','HADM_ID','ICUSTAY_ID','ITEMID','CHARTTIME','VALUENUM','VALUEUOM']
chart_chunks = pd.read_csv(
    f"{base_dir}/CHARTEVENTS.csv",
    usecols=use_cols,
    chunksize=1_000_000,
    low_memory=False,
    dtype={
        'SUBJECT_ID':'Int64',
        'HADM_ID':'string',
        'ICUSTAY_ID':'Int64',
        'ITEMID':'Int64',
        'VALUEUOM':'string'
    }
)

In [47]:
vital_map = vital_itemids.set_index('ITEMID')['VITAL']

In [48]:
# F to C conversion
def convert_temp_to_c(row):
    # Convert Fahrenheit to Celsius only for TEMP_F
    if row['VITAL'] == 'TEMP_F' and pd.notna(row['VALUENUM']):
        return (row['VALUENUM'] - 32.0) * (5.0/9.0)
    return row['VALUENUM']

In [49]:
collected = []

for chunk in chart_chunks:
    # Keep only necessary ITEMIDs
    chunk = chunk[chunk['ITEMID'].isin(vital_map.index)]
    if chunk.empty:
        continue

    # Attach canonical vital name
    chunk['VITAL'] = chunk['ITEMID'].map(vital_map)

    # Times and 24h window
    chunk['CHARTTIME'] = pd.to_datetime(chunk['CHARTTIME'], errors='coerce')
    merged = chunk.merge(icu_keys, on='ICUSTAY_ID', how='inner')
    win = merged[(merged['CHARTTIME'] >= merged['INTIME']) & (merged['CHARTTIME'] <= merged['INTIME_24H'])]
    if win.empty:
        continue

    # Numeric first/only
    win = win[pd.to_numeric(win['VALUENUM'], errors='coerce').notna()]
    win['VALUENUM'] = win['VALUENUM'].astype(float)

    # Temperature normalization first
    win.loc[win['VITAL'] == 'TEMP_F', 'VALUENUM'] = (win.loc[win['VITAL'] == 'TEMP_F', 'VALUENUM'] - 32.0) * (5.0/9.0)
    win.loc[win['VITAL'] == 'TEMP_F', 'VITAL'] = 'TEMP_C'

    # Physiological bounds (safety measure for vitals without bounds)
    bounds = pd.DataFrame([
        ('HEART_RATE', 20, 250),
        ('RESP_RATE',   4, 100),
        ('SBP',        40, 300),
        ('DBP',        20, 200),
        ('MAP',        30, 250),
        ('TEMP_C',     30, 43),
        ('SPO2',       50, 100),
    ], columns=['VITAL','LOW','HIGH'])

    # Remove impossible values
    win = win.merge(bounds, on='VITAL', how='left')
    ok_low  = win['LOW'].isna()  | (win['VALUENUM'] >= win['LOW'])
    ok_high = win['HIGH'].isna() | (win['VALUENUM'] <= win['HIGH'])
    win = win[ok_low & ok_high]

    # Aggregate per ICU stay and vital
    collected.append(
        win.groupby(['ICUSTAY_ID','VITAL'])['VALUENUM'].agg(['mean','min','max']).reset_index()
    )

In [50]:
# Combine chunk aggregates, then aggregate again to be safe
if collected:
    vit_agg = pd.concat(collected, ignore_index=True)
    vit_agg = vit_agg.groupby(['ICUSTAY_ID','VITAL'])[['mean','min','max']].mean().reset_index()
else:
    vit_agg = pd.DataFrame(columns=['ICUSTAY_ID','VITAL','mean','min','max'])
    
# Pivot to columns
vit_wide = vit_agg.pivot(index='ICUSTAY_ID', columns='VITAL')
vit_wide.columns = [f"{v}_{stat}" for stat, v in vit_wide.columns]
vit_wide = vit_wide.reset_index()
vit_wide.head()

,ICUSTAY_ID,DBP_mean,HEART_RATE_mean,MAP_mean,RESP_RATE_mean,SBP_mean,SPO2_mean,TEMP_C_mean,DBP_min,HEART_RATE_min,...,SBP_min,SPO2_min,TEMP_C_min,DBP_max,HEART_RATE_max,MAP_max,RESP_RATE_max,SBP_max,SPO2_max,TEMP_C_max
0,200001,57.346154,99.275862,68.384615,21.230769,103.423077,96.705882,37.175926,49.0,60.0,...,89.0,85.000000,36.388889,68.0,134.0,79.0,32.0,118.0,100.000000,37.666667
1,200003,64.062500,101.446429,130.343750,29.775000,112.000000,96.025000,37.361107,49.0,81.0,...,89.0,93.500000,36.416645,81.0,121.0,142.0,38.5,141.0,97.250000,38.305555
2,200006,NaN,73.096491,NaN,18.888889,NaN,96.009524,37.020836,NaN,65.5,...,NaN,94.800000,36.638883,NaN,79.5,NaN,24.0,NaN,96.600000,37.388906
3,200007,NaN,94.064516,NaN,21.520000,NaN,95.750000,37.188884,NaN,80.0,...,NaN,94.000000,36.388889,NaN,104.0,NaN,29.0,NaN,97.000000,37.611109
4,200009,62.645161,95.774194,74.741935,15.861111,101.161290,97.282828,37.245161,49.0,88.0,...,86.0,95.666667,34.599998,84.0,106.0,101.0,22.0,130.0,97.333333,38.300001


Merge Vitals into the rest and save

In [51]:
cohort_v = cohort.merge(vit_wide, on='ICUSTAY_ID', how='left')

In [52]:
# Save
out_csv = os.path.join(base_dir, "cohort_labs_vitals24h.csv")
cohort_v.to_csv(out_csv, index=False)
cohort_v.shape

(61532, 54)

In [53]:
feat_cols = [c for c in cohort_v.columns if c.endswith(('_mean','_min','_max'))]
coverage = cohort_v[feat_cols].notna().mean().sort_values(ascending=False)
coverage.head(10), coverage.tail(10)

cohort_v['HOSPITAL_EXPIRE_FLAG'].value_counts(normalize=True)

HOSPITAL_EXPIRE_FLAG
0    0.892592
1    0.107408
Name: proportion, dtype: float64

Freezing the Master Table

In [54]:
# Labs and vitals merged
master = cohort_v.copy()  
master.to_csv(f"{base_dir}\MERGED\master_24h.csv", index=False)